In [20]:
from datetime import date
import requests
import urllib
from bs4 import BeautifulSoup
import json
import os
import csv
import pandas as pd
import glob

#today = date.today()


# mm/dd/y
today_YYYYMMDD = date.today().strftime("%Y-%m-%d")
print("today is:", today_YYYYMMDD)



today is: 2022-06-03


In [99]:
#Find Last Update Date

list_of_previous_updates = [file for file in sorted(glob.glob('PDB_list_CSVs/*-additions.txt'))]  #for full set, look in subdirs 

last_update = list_of_previous_updates[-1][14:24] #"2022-04-11"

print("last PDB database update was:", last_update)


last PDB database update was: 2022-06-03


In [100]:
#This is the actual query that works, { -> %3A, : -> %7B etc. I couldn't get url encoding to match exct yet.

#last_update is date pulled from a file name; inserted into query
#eg. last_update = "2022-04-11"

my_json_query_str = f'%7B"query"%3A%7B"type"%3A"group"%2C"logical_operator"%3A"and"%2C"nodes"%3A%5B%7B"type"%3A"terminal"%2C"service"%3A"text"%2C"parameters"%3A%7B"attribute"%3A"exptl.method"%2C"operator"%3A"exact_match"%2C"negation"%3Afalse%2C"value"%3A"X-RAY%20DIFFRACTION"%7D%7D%2C%7B"type"%3A"terminal"%2C"service"%3A"text"%2C"parameters"%3A%7B"attribute"%3A"rcsb_accession_info.initial_release_date"%2C"operator"%3A"greater"%2C"negation"%3Afalse%2C"value"%3A"{last_update}"%7D%7D%5D%2C"label"%3A"text"%7D%2C"return_type"%3A"entry"%2C"request_options"%3A%7B"paginate"%3A%7B"start"%3A0%2C"rows"%3A100000%7D%2C"scoring_strategy"%3A"combined"%2C"sort"%3A%5B%7B"sort_by"%3A"score"%2C"direction"%3A"desc"%7D%5D%7D%7D'

'%7B"query"%3A%7B"type"%3A"group"%2C"logical_operator"%3A"and"%2C"nodes"%3A%5B%7B"type"%3A"terminal"%2C"service"%3A"text"%2C"parameters"%3A%7B"attribute"%3A"exptl.method"%2C"operator"%3A"exact_match"%2C"negation"%3Afalse%2C"value"%3A"X-RAY%20DIFFRACTION"%7D%7D%2C%7B"type"%3A"terminal"%2C"service"%3A"text"%2C"parameters"%3A%7B"attribute"%3A"rcsb_accession_info.initial_release_date"%2C"operator"%3A"greater"%2C"negation"%3Afalse%2C"value"%3A"2022-06-03"%7D%7D%5D%2C"label"%3A"text"%7D%2C"return_type"%3A"entry"%2C"request_options"%3A%7B"paginate"%3A%7B"start"%3A0%2C"rows"%3A100000%7D%2C"scoring_strategy"%3A"combined"%2C"sort"%3A%5B%7B"sort_by"%3A"score"%2C"direction"%3A"desc"%7D%5D%7D%7D'

In [7]:
#Query PDB using REST API
#Search for X-Ray structures deposited since last date

page = requests.get(f"https://search.rcsb.org/rcsbsearch/v2/query?json={my_json_query_str}")
print(f"https://search.rcsb.org/rcsbsearch/v2/query?json={my_json_query_str}")

https://search.rcsb.org/rcsbsearch/v2/query?json=%7B"query"%3A%7B"type"%3A"group"%2C"logical_operator"%3A"and"%2C"nodes"%3A%5B%7B"type"%3A"terminal"%2C"service"%3A"text"%2C"parameters"%3A%7B"attribute"%3A"exptl.method"%2C"operator"%3A"exact_match"%2C"negation"%3Afalse%2C"value"%3A"X-RAY%20DIFFRACTION"%7D%7D%2C%7B"type"%3A"terminal"%2C"service"%3A"text"%2C"parameters"%3A%7B"attribute"%3A"rcsb_accession_info.initial_release_date"%2C"operator"%3A"greater"%2C"negation"%3Afalse%2C"value"%3A"2022-04-11"%7D%7D%5D%2C"label"%3A"text"%7D%2C"return_type"%3A"entry"%2C"request_options"%3A%7B"paginate"%3A%7B"start"%3A0%2C"rows"%3A100000%7D%2C"scoring_strategy"%3A"combined"%2C"sort"%3A%5B%7B"sort_by"%3A"score"%2C"direction"%3A"desc"%7D%5D%7D%7D


In [8]:
#Parse response from PDB
soup = BeautifulSoup(page.text, "html")
jason = json.loads(soup.body.get_text())

json.dumps(jason)

'{"query_id": "72580854-197b-49dd-b594-30f9297b0d25", "result_type": "entry", "total_count": 1375, "result_set": [{"identifier": "5S9L", "score": 1.0}, {"identifier": "5S9M", "score": 1.0}, {"identifier": "5S9N", "score": 1.0}, {"identifier": "5SB3", "score": 1.0}, {"identifier": "5SB4", "score": 1.0}, {"identifier": "5SB5", "score": 1.0}, {"identifier": "5SB6", "score": 1.0}, {"identifier": "5SB7", "score": 1.0}, {"identifier": "6YMC", "score": 1.0}, {"identifier": "6Z1B", "score": 1.0}, {"identifier": "6Z48", "score": 1.0}, {"identifier": "6ZH0", "score": 1.0}, {"identifier": "7AG0", "score": 1.0}, {"identifier": "7AKK", "score": 1.0}, {"identifier": "7AKZ", "score": 1.0}, {"identifier": "7AL7", "score": 1.0}, {"identifier": "7ALE", "score": 1.0}, {"identifier": "7AMA", "score": 1.0}, {"identifier": "7AMG", "score": 1.0}, {"identifier": "7AMP", "score": 1.0}, {"identifier": "7AMQ", "score": 1.0}, {"identifier": "7AMR", "score": 1.0}, {"identifier": "7AMS", "score": 1.0}, {"identifier

In [101]:
#Generate list of PDB names to populate CSV file

csv_list = [] 
for mini_dic in jason['result_set']:
    csv_list.append(mini_dic["identifier"])

In [12]:
#Write CSV File containing IDs of new PBB additions

#import csv
with open(f'./PDB_list_CSVs/{today_YYYYMMDD}-new-PDB-additions.txt', 'w') as f:
    write = csv.writer(f)  
    write.writerow(csv_list)
    
#Write file name with list of new PDBs to Bash Environment
os.environ['PDB_LIST'] = f'./PDB_list_CSVs/{today_YYYYMMDD}-new-PDB-additions.txt'

In [16]:
%%bash

#Download the new PDBs
#Failures apprear to be files published but not yet released 
./PDB_batch_download.sh -f $PDB_LIST -o ./PDB_depot -p


Failed to download https://files.rcsb.org/download/7AYY.pdb.gz
Failed to download https://files.rcsb.org/download/7AYZ.pdb.gz
Failed to download https://files.rcsb.org/download/7AZ0.pdb.gz
Failed to download https://files.rcsb.org/download/7AZO.pdb.gz
Failed to download https://files.rcsb.org/download/7EW5.pdb.gz
Failed to download https://files.rcsb.org/download/7EW8.pdb.gz
Failed to download https://files.rcsb.org/download/7F25.pdb.gz
Failed to download https://files.rcsb.org/download/7MFN.pdb.gz
Failed to download https://files.rcsb.org/download/7MFS.pdb.gz
Failed to download https://files.rcsb.org/download/7MHE.pdb.gz
Failed to download https://files.rcsb.org/download/7MKR.pdb.gz
Failed to download https://files.rcsb.org/download/7MKS.pdb.gz
Failed to download https://files.rcsb.org/download/7MQT.pdb.gz
Failed to download https://files.rcsb.org/download/7MX6.pdb.gz
Failed to download https://files.rcsb.org/download/7MXZ.pdb.gz
Failed to download https://files.rcsb.org/download/7MY1

In [27]:
%%bash

#unzip the new files
gunzip -d ./PDB_depot/*pdb.gz

Process is interrupted.


In [75]:
#Populate dataframe of new PDB info
df_new = pd.DataFrame(columns=["Resolution", "Completeness", "I_sigma", "R_value", "R_free", "Detector", "Det_type", "Optics"])


#Add date - I can drop anything before... 2002? 20 years is a lot and the early entries are garbage/useless

                                    #only the new PDBs (not yet moved to subdirs)
for filename in glob.glob('PDB_depot/*.pdb'):  #for full set, look in subdirs 
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        resolution = completeness = i_sigma = r_value = r_free = detector = det_type = optics = "NULL"  #set as null in case line not present
        
        pdb_id = filename[-8:-4]
        for line in f:

            #Exit condition, 50% drop in time
            #works without strip? remove other line strips for speed 30% drop in time
            if line.startswith("ATOM"):
                break
            
            #Resolution
            if line.startswith("REMARK   3   RESOLUTION RANGE HIGH (ANGSTROMS)"):
                resolution = line.split(":")[1].strip()

            #Completeness
            if line.startswith("REMARK   3   COMPLETENESS FOR RANGE        (%)"):
                completeness = line.split(":")[1].strip()         
                
            #I/sigma
            if line.startswith("REMARK 200  <I/SIGMA(I)> FOR THE DATA SET"):
                i_sigma = line.split(":")[1].strip()      
            
            #R-Value
            if line.startswith("REMARK   3   R VALUE     (WORKING + TEST SET)"):
                r_value = line.split(":")[1].strip()
            
            
            #R-Free
            if line.startswith("REMARK   3   FREE R VALUE"):
                if line.split(":")[0].strip().endswith("VALUE"):  #need to match end here
                    r_free = line.split(":")[1].strip()
                    
                    
            #Detector Model (Pilatus 6M, Eiger 16M etc)
            if line.startswith("REMARK 200  DETECTOR MANUFACTURER"):
                detector = line.split(":")[1].strip()
                if ";" in detector:
                    detector = line.split(';')[0].strip()  #some lines contain 2 fields, 2nd redundant
                
            #Detector Type (Pixel, CCD, etc)
            if line.startswith("REMARK 200  DETECTOR TYPE"):
                det_type = line.split(":")[1].strip()

                    
            #Optics (Mirrors)
            if line.startswith("REMARK 200  OPTICS"):
                optics = line.split(":")[1].strip()

           
        
        #print(pdb_id, resolution, completeness, i_sigma, r_value, r_free, detector, det_type, optics)
        df_new.loc[pdb_id] = [resolution, completeness, i_sigma, r_value, r_free, detector, det_type, optics]


In [76]:
#Load the existing (full) dataframe of PDB info
df_full = pd.read_feather('/Users/jesse/TDI_bootcamp/capstone_PDB/dataframe_feather/1_initial_dataframe_163K_full.feather')

In [79]:
#Concat old and new dataframe
df_catted = pd.concat([df_new.reset_index(), df_full]) 

In [80]:
df_catted = df_catted.reset_index().drop(['level_0'], axis=1) #remove old indices column

In [81]:
#Write the new, updated PDB DF to feather

df_catted.to_feather('/Users/jesse/TDI_bootcamp/capstone_PDB/dataframe_feather/1_updated_dataframe.feather')

In [85]:
%%bash
#Move the new PDB files into their respective subdirectories

for n in $(seq 1 9); do
    for i in `ls PDB_depot/${n}*.pdb`; do    #backticks don't actually need curly braces for $VAR expansion
        mv $i  "PDB_depot/${n}xxx_pdbs"; #Double quotes do
    done
    done

ls: PDB_depot/1*.pdb: No such file or directory
ls: PDB_depot/2*.pdb: No such file or directory
ls: PDB_depot/3*.pdb: No such file or directory
ls: PDB_depot/4*.pdb: No such file or directory
ls: PDB_depot/9*.pdb: No such file or directory


1xxx_pdbs                         7QT7.pdb
2xxx_pdbs                         7QT8.pdb
3xxx_pdbs                         7QT9.pdb
4xxx_pdbs                         7QUT.pdb
5S9L.pdb                          7QVZ.pdb
5S9M.pdb                          7QWK.pdb
5S9N.pdb                          7QWT.pdb
5SB3.pdb                          7QXM.pdb
5SB4.pdb                          7QXY.pdb
5SB5.pdb                          7QXZ.pdb
5SB6.pdb                          7QY0.pdb
5SB7.pdb                          7QY1.pdb
5xxx_pdbs                         7QY2.pdb
6YMC.pdb                          7QYN.pdb
6Z1B.pdb                          7QZQ.pdb
6Z48.pdb                          7R02.pdb
6ZH0.pdb                          7R0A.pdb
6xxx_pdbs                         7R0M.pdb
7AG0.pdb                          7R0N.pdb
7AKK.pdb                          7R0Q.pdb
7AKZ.pdb                          7R1K.pdb
7AL7.pdb                          7R1Z.pdb
7ALE.pdb                          7R20.pdb
7AMA.pdb   